In [87]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

In [88]:
df = pd.read_csv('../data/icu_ibd_all_table.csv')
df['intime'] = pd.to_datetime(df['intime'])
df.shape

(1161, 32)

In [89]:
df2 = pd.read_csv('../data/patients_ibd.csv')


func = lambda x: pd.to_datetime('{}-01-01'.format(x))
df2['anchor_year'] = df2['anchor_year'].apply(func)

df2.shape

(2417, 6)

In [118]:
data = df.merge(df2[['subject_id', 'anchor_year']], on='subject_id', how='left')
data['age'] = ((data['intime'] - data['anchor_year']).dt.days) / 365 + data['age']

In [119]:
for col in data.columns:
    print(col, end=' ')
    print(data[col].isna().sum(), end=' - ')
    print(len(data[col].unique()))

hadm_id 0 - 1161
subject_id 0 - 683
intime 0 - 1161
outtime 0 - 1161
los 0 - 1159
mortality 0 - 2
gender 0 - 2
age 0 - 1128
weight 1066 - 28
bmi 1070 - 25
heart_rate 1 - 108
systolic_pressure 1099 - 17
diastolic_pressure 1099 - 15
respiratory_rate 1 - 37
temperature 962 - 45
white_blood_cell 1155 - 5
hematocrit 18 - 252
red_blood_cell 705 - 68
rdw 17 - 122
platelet 18 - 454
mcv 17 - 57
mch 17 - 157
hemoglobin 16 - 102
CRP 1161 - 1
race 18 - 25
language 18 - 3
marital_status 32 - 5
insurance 18 - 4
die_in_icu 0 - 2
uc_only 0 - 2
cd_only 0 - 2
uc_cd 0 - 2
anchor_year 0 - 93


In [120]:
cols_except = [
    'hadm_id',
    'subject_id',
    'intime',
    'outtime',
    # 'los',
    'mortality',
    'weight',
    'bmi',
    'systolic_pressure',
    'diastolic_pressure',
    'temperature',
    'white_blood_cell',
    'red_blood_cell',
    'CRP',
    'die_in_icu',
    'anchor_year',
    'uc_cd'
]

cols_include = [col for col in data.columns if col not in cols_except]

data = data[cols_include]

In [121]:
X = data[ ~data[cols_include].isna().any(axis=1) ]


def parse_race(race):
    if 'WHITE' in race:
        return 'WHITE'
    elif 'BLACK' in race:
        return 'BLACK'
    elif 'HISPANIC' in race or 'LATINO' in race:
        return 'HISPANIC/LATINO'
    elif 'ASIAN' in race:
        return 'ASIAN'
    else:
        return 'OTHER'


X.loc[:, 'race'] = X['race'].apply(parse_race)

for col in ['language', 'race', 'marital_status', 'insurance']:
    print(X[col].value_counts())

X = pd.get_dummies(X, columns=['race', 'language', 'marital_status', 'insurance'], prefix=['race', 'language', 'marital_status', 'insurance'])

language
ENGLISH    1056
?            52
Name: count, dtype: int64
race
WHITE              918
BLACK              101
OTHER               61
HISPANIC/LATINO     21
ASIAN                7
Name: count, dtype: int64
marital_status
MARRIED     510
SINGLE      367
WIDOWED     136
DIVORCED     95
Name: count, dtype: int64
insurance
Medicare    516
Other       505
Medicaid     87
Name: count, dtype: int64


In [122]:
cols_min_max = ['age', 'heart_rate', 'respiratory_rate', 'hematocrit', 'rdw',
       'platelet', 'mcv', 'mch', 'hemoglobin']

tmp = X.loc[:, cols_min_max]

tmp = ( tmp - tmp.min() ) / ( tmp.max() - tmp.min() )

X.loc[:, col_min_max] = tmp

In [123]:
X.loc[:, [col for col in X.columns if col not in cols_min_max+['los']]] = \
    X.loc[:, [col for col in X.columns if col not in cols_min_max+['los']]].astype(int)

C:\Users\sitdo\AppData\Local\Temp\ipykernel_27556\164024701.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0 0 0 ... 0 0 0]' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  X.loc[:, [col for col in X.columns if col not in cols_min_max+['los']]] = \
C:\Users\sitdo\AppData\Local\Temp\ipykernel_27556\164024701.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0 0 0 ... 0 0 0]' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  X.loc[:, [col for col in X.columns if col not in cols_min_max+['los']]] = \
C:\Users\sitdo\AppData\Local\Temp\ipykernel_27556\164024701.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0 0 0 ... 0 0 0]' has dtype incompatible with bool, please explicitly cast to a c

In [127]:
X.to_csv('../new r scripts/data.csv')